### Fit
 + https://pytorch.org/tutorials/beginner/nn_tutorial.html

In [ ]:
%matplotlib inline

import sys
import logging
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch import nn
from torch import optim

sys.path.append('..')
import utils

logging.basicConfig(level=logging.INFO)

### Test relu

In [ ]:
x = range(-5,6)
plt.plot(x, F.relu(torch.tensor(x)).data.numpy(), 'bo-')
plt.grid(True)

### Get data

In [ ]:
data_dir = '../data'
x_train, y_train, x_valid, y_valid = utils.get_mnist(data_dir)

bs = 64

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs)

### Define architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

### Fit

In [ ]:
model = Net()
opt = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
loss_func = F.cross_entropy

utils.fit(2, model, loss_func, opt, train_dl, valid_dl)